In [19]:
!ls

backbone.py	 external	  notebook3.ipynb   scripts	 weights
CHANGELOG.md	 layers		  notebook.ipynb    train_o1.py  yolact_cpu.py
data		 LICENSE	  __pycache__	    train.py	 yolact.py
environment.yml  logs		  README.md	    utils
eval.py		 notebook2.ipynb  run_coco_eval.py  web


In [20]:
# # Clone the official YOLACT repository
# !git clone https://github.com/dbolya/yolact.git
# %cd yolact

# # Install required packages
# !pip install torch torchvision opencv-python Pillow pycocotools matplotlib

In [21]:
# # Create the weights directory
# !mkdir -p weights

# # Download the pre-trained backbone weights for ResNet-101
# !wget https://s3.amazonaws.com/images.criterion.com/models/yolact/resnet101_reducedfc.pth -P weights/
# print("Backbone weights downloaded.")

In [22]:
# This cell contains the full, CPU-safe model definition from your yolact_cpu.py file,
# AND a patch for the loss function to prevent the RuntimeError.

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import Bottleneck
import numpy as np
from itertools import product
from math import sqrt
from typing import List
from collections import defaultdict
from data.config import cfg, mask_type, activation_func
from layers import Detect
from layers.interpolate import InterpolateModule
from backbone import construct_backbone
import torch.backends.cudnn as cudnn
from utils import timer
from utils.functions import MovingAverage, make_net
from yolact import Yolact, FPN, PredictionModule

import importlib.util
import sys

# ------------------- PATCH FOR multibox_loss.py -------------------
# This patch fixes a numerical instability issue in the original loss function.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# print("Applying numerical stability patch to MultiBoxLoss...")

# file_path = 'layers/modules/multibox_loss.py'
# module_name = 'layers.modules.multibox_loss'

# with open(file_path, 'r') as f:
#     multibox_loss_code = f.read()

# # Replace the unstable BCE loss with the stable BCEWithLogitsLoss
# # 1. Remove the explicit sigmoid activation on the predicted masks
# original_line_1 = 'pred_masks = cfg.mask_proto_mask_activation(pred_masks)'
# replacement_1 = '# pred_masks = cfg.mask_proto_mask_activation(pred_masks) # Patched out for numerical stability'
# multibox_loss_code_fixed = multibox_loss_code.replace(original_line_1, replacement_1)

# # 2. Replace the BCE function with the version that takes logits
# original_line_2 = 'pre_loss = F.binary_cross_entropy(torch.clamp(pred_masks, 0, 1), mask_t, reduction=\'none\')'
# replacement_2 = 'pre_loss = F.binary_cross_entropy_with_logits(pred_masks, mask_t, reduction=\'none\')'
# multibox_loss_code_fixed = multibox_loss_code_fixed.replace(original_line_2, replacement_2)

# # Load the patched code as a module
# spec = importlib.util.spec_from_loader(module_name, loader=None, origin=file_path)
# multibox_loss_module = importlib.util.module_from_spec(spec)
# exec(multibox_loss_code_fixed, multibox_loss_module.__dict__)
# sys.modules[module_name] = multibox_loss_module

# print("Patch applied successfully.")
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# --------------------------------------------------------------------

# Now that the patched module is loaded, we can import from it
from layers.modules.multibox_loss import MultiBoxLoss


# This is the corrected CUDA initialization from your yolact_cpu.py
# if torch.cuda.is_available():
#     torch.cuda.current_device()
# else:
#     print("⚠️ CUDA not available, running on CPU.")

# # --- The full, corrected code from yolact_cpu.py starts here ---
# # (This is the same code as the previous step, defining Yolact and its dependencies)
# use_jit = torch.cuda.device_count() <= 1
# if not use_jit:
#     print('Multiple GPUs detected! Turning off JIT.')

# ScriptModuleWrapper = torch.jit.ScriptModule if use_jit else nn.Module
# # ... (The rest of the full Yolact class and its dependencies (PredictionModule, FPN, etc.) go here)
# class Yolact(nn.Module):
#     # ... (Full Yolact class definition from previous step)
#     def __init__(self):
#         super().__init__()
#         self.backbone = construct_backbone(cfg.backbone)
#         if cfg.freeze_bn: self.freeze_bn()
#         if cfg.mask_type == mask_type.direct: cfg.mask_dim = cfg.mask_size**2
#         elif cfg.mask_type == mask_type.lincomb:
#             if cfg.mask_proto_use_grid:
#                 self.grid = torch.Tensor(np.load(cfg.mask_proto_grid_file))
#                 self.num_grids = self.grid.size(0)
#             else: self.num_grids = 0
#             self.proto_src = cfg.mask_proto_src
#             if self.proto_src is None: in_channels = 3
#             elif cfg.fpn is not None: in_channels = cfg.fpn.num_features
#             else: in_channels = self.backbone.channels[self.proto_src]
#             in_channels += self.num_grids
#             self.proto_net, cfg.mask_dim = make_net(in_channels, cfg.mask_proto_net, include_last_relu=False)
#             if cfg.mask_proto_bias: cfg.mask_dim += 1
#         self.selected_layers = cfg.backbone.selected_layers
#         src_channels = self.backbone.channels
#         if cfg.use_maskiou: self.maskiou_net = FastMaskIoUNet()
#         if cfg.fpn is not None:
#             self.fpn = FPN([src_channels[i] for i in self.selected_layers])
#             self.selected_layers = list(range(len(self.selected_layers) + cfg.fpn.num_downsample))
#             src_channels = [cfg.fpn.num_features] * len(self.selected_layers)
#         self.prediction_layers = nn.ModuleList()
#         cfg.num_heads = len(self.selected_layers)
#         for idx, layer_idx in enumerate(self.selected_layers):
#             parent = self.prediction_layers[0] if cfg.share_prediction_module and idx > 0 else None
#             pred = PredictionModule(src_channels[layer_idx], src_channels[layer_idx],
#                                     aspect_ratios = cfg.backbone.pred_aspect_ratios[idx],
#                                     scales        = cfg.backbone.pred_scales[idx],
#                                     parent=parent, index=idx)
#             self.prediction_layers.append(pred)
#         if cfg.use_class_existence_loss:
#             self.class_existence_fc = nn.Linear(src_channels[-1], cfg.num_classes - 1)
#         if cfg.use_semantic_segmentation_loss:
#             self.semantic_seg_conv = nn.Conv2d(src_channels[0], cfg.num_classes-1, kernel_size=1)
#         self.detect = Detect(cfg.num_classes, bkg_label=0, top_k=cfg.nms_top_k, conf_thresh=cfg.nms_conf_thresh, nms_thresh=cfg.nms_thresh)
#     def save_weights(self, path): torch.save(self.state_dict(), path)
#     def load_weights(self, path, device='cpu'): # Added device parameter
#         state_dict = torch.load(path, map_location=device) 
#         self.load_state_dict(state_dict)
#     def init_weights(self, backbone_path):
#         self.backbone.init_backbone(backbone_path)
#         for name, module in self.named_modules():
#             is_conv_layer = isinstance(module, nn.Conv2d)
#             if is_conv_layer and module not in self.backbone.backbone_modules:
#                 nn.init.xavier_uniform_(module.weight.data)
#                 if module.bias is not None:
#                     module.bias.data.zero_()
#     def train(self, mode=True):
#         super().train(mode)
#         if cfg.freeze_bn: self.freeze_bn()
#     def freeze_bn(self, enable=False):
#         for module in self.modules():
#             if isinstance(module, nn.BatchNorm2d):
#                 module.train() if enable else module.eval()
#                 module.weight.requires_grad = enable
#                 module.bias.requires_grad = enable
#     def forward(self, x):
#         _, _, img_h, img_w = x.size()
#         cfg._tmp_img_h, cfg._tmp_img_w = img_h, img_w
#         outs = self.backbone(x)
#         if cfg.fpn is not None:
#             outs = [outs[i] for i in cfg.backbone.selected_layers]
#             outs = self.fpn(outs)
#         proto_out = None
#         if cfg.mask_type == mask_type.lincomb and cfg.eval_mask_branch:
#             proto_x = x if self.proto_src is None else outs[self.proto_src]
#             proto_out = self.proto_net(proto_x)
#             proto_out = cfg.mask_proto_prototype_activation(proto_out)
#             proto_out = proto_out.permute(0, 2, 3, 1).contiguous()
#             if cfg.mask_proto_bias:
#                 bias_shape = [x for x in proto_out.size()]
#                 bias_shape[-1] = 1
#                 proto_out = torch.cat([proto_out, torch.ones(*bias_shape, device=x.device)], -1)
#         pred_outs = { 'loc': [], 'conf': [], 'mask': [], 'priors': [] }
#         if cfg.use_mask_scoring: pred_outs['score'] = []
#         if cfg.use_instance_coeff: pred_outs['inst'] = []
#         for idx, pred_layer in zip(self.selected_layers, self.prediction_layers):
#             p = pred_layer(outs[idx])
#             for k, v in p.items(): pred_outs[k].append(v)
#         for k, v in pred_outs.items():
#             pred_outs[k] = torch.cat(v, -2)
#         if proto_out is not None: pred_outs['proto'] = proto_out
#         if self.training:
#             if cfg.use_semantic_segmentation_loss:
#                 pred_outs['segm'] = self.semantic_seg_conv(outs[0])
#             return pred_outs
#         else:
#             pred_outs['conf'] = F.softmax(pred_outs['conf'], -1)
#             return self.detect(pred_outs, self)
# --- Training Helper Class ---
class NetLoss(nn.Module):
    def __init__(self, net:Yolact, criterion:MultiBoxLoss):
        super().__init__()
        self.net = net
        self.criterion = criterion
    
    def forward(self, images, targets, masks, num_crowds):
        preds = self.net(images)
        losses = self.criterion(self.net, preds, targets, masks, num_crowds)
        return losses


#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# --- A simple class to mimic the command-line arguments ---
class Args:
    def __init__(self, use_cuda):
        self.batch_size = 4
        self.resume = None
        self.start_iter = -1
        self.num_workers = 0
        self.cuda = use_cuda
        self.lr = 1e-3
        self.momentum = 0.9
        self.decay = 5e-4
        self.gamma = 0.1
        self.save_folder = 'weights/'
        self.log_folder = 'logs/'
        self.config = None
        self.save_interval = 200 # Set a lower save interval for the short test
        self.validation_epoch = -1 # Disable validation for this simple example
        self.keep_latest = True
        self.log = True

print("YOLACT model and helper classes defined.")


YOLACT model and helper classes defined.


In [23]:
from data.config import Config, yolact_base_config, resnet50_backbone, mask_type, fpn_base

# ----------------- CONFIGURATION -----------------
# Set this to True to use a GPU, or False to use the CPU.
USE_CUDA = False
# -----------------------------------------------

# Define your custom dataset paths and class names
dataset_base = Config({
    'name': 'My Dataset',
    'train_images': 'data/my_dataset/train',
    'train_info':   'data/my_dataset/train/_annotations.coco.json',
    'valid_images': 'data/my_dataset/valid',
    'valid_info':   'data/my_dataset/valid/_annotations.coco.json',
    'class_names': ('bench', 'chair', 'couch', 'dining table', 'laptop', 'person'),
    'has_gt': True,      # FIX: This was missing
    'label_map': None,   # FIX: This was missing
})

# Create your custom config by overriding the base_config
yolact_my_config = yolact_base_config.copy({
    'name': 'yolact_custom',
    # FIX: Use the imported resnet50_backbone configuration
    'backbone': resnet50_backbone.copy({
        'selected_layers': list(range(1, 4)),
        'pred_scales': [[24], [48], [96], [192], [384]],
        'pred_aspect_ratios': [ [[1, 1/2, 2]] ]*5,
        'use_pixel_scales': True,
        'preapply_sqrt': False,
        'use_square_anchors': True,
}),
    # 'backbone': resnet50_backbone.copy(),
    'dataset': dataset_base,
    'num_classes': len(dataset_base.class_names) + 1,
    'max_iter': 10,     # Using a low number for a quick test run
    'lr_steps': (120, 160, 180),
    # 'lr_steps': (120, 160, 180),
})

# --- Activate Configuration ---
cfg.replace(yolact_my_config)
# Add the missing debug attribute to prevent the AttributeError in postprocess()
cfg.mask_proto_debug = False
print("Configuration is set.")
print(f"Number of classes: {cfg.num_classes}")
print(f"Using CUDA: {USE_CUDA}")

Configuration is set.
Number of classes: 7
Using CUDA: False


In [24]:
import torch.optim as optim
import torch.utils.data as data
import datetime
import os
from data import COCODetection, detection_collate
from utils.augmentations import SSDAugmentation

def train():
    args = Args(use_cuda=USE_CUDA)
    
    if not os.path.exists(args.save_folder):
        os.mkdir(args.save_folder)
        
    device = torch.device('cuda' if args.cuda and torch.cuda.is_available() else 'cpu')
    
    if args.cuda and torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        torch.set_default_tensor_type('torch.FloatTensor')

    dataset = COCODetection(image_path=cfg.dataset.train_images,
                            info_file=cfg.dataset.train_info,
                            transform=SSDAugmentation())

    yolact_net = Yolact().to(device)
    yolact_net.train()

    print('Initializing weights...')
    yolact_net.init_weights(backbone_path=args.save_folder + cfg.backbone.path)

    optimizer = optim.SGD(yolact_net.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.decay)
    criterion = MultiBoxLoss(num_classes=cfg.num_classes,
                             pos_threshold=cfg.positive_iou_threshold,
                             neg_threshold=cfg.negative_iou_threshold,
                             negpos_ratio=cfg.ohem_negpos_ratio)
    
    net = NetLoss(yolact_net, criterion).to(device)

    data_loader = data.DataLoader(dataset, args.batch_size,
                                  num_workers=args.num_workers,
                                  shuffle=True, collate_fn=detection_collate,
                                  pin_memory=False)

    iteration = 0
    print('\nBegin training!')
    for epoch in range(100):
        if iteration >= cfg.max_iter:
            break
            
        for datum in data_loader:
            if iteration >= cfg.max_iter:
                break

            images, (targets, masks, num_crowds) = datum
            
            # Move data to the correct device
            images = torch.stack(images, 0).to(device)
            targets = [ann.to(device) for ann in targets]
            masks = [mask.to(device) for mask in masks]

            optimizer.zero_grad()
            
            losses = net(images, targets, masks, num_crowds)
            
            losses = { k: v.mean() for k, v in losses.items() }
            loss = sum(losses.values())
            
            loss.backward()
            optimizer.step()

            if iteration % 10 == 0:
                print(f'[{iteration:5d}] Loss: {loss.item():.4f}')

            if iteration % args.save_interval == 0 and iteration != 0:
                print(f'Saving state, iter: {iteration}')
                yolact_net.save_weights(f'{args.save_folder}/yolact_custom_{iteration}.pth')
            
            iteration += 1
            
    yolact_net.save_weights(f'{args.save_folder}/yolact_custom_final.pth')
    print("Training finished.")



In [25]:
# Start the training process
train()

loading annotations into memory...


Done (t=0.02s)
creating index...
index created!
Initializing weights...

Begin training!
[    0] Loss: 1750.6454
Training finished.


In [26]:
# This cell contains all the necessary helper classes and functions from eval.py,
# adapted to work within this notebook.

import pycocotools
from layers.box_utils import jaccard, mask_iou
from utils.functions import ProgressBar
from collections import OrderedDict
from utils.functions import MovingAverage, ProgressBar

# --- Helper Classes and Functions from eval.py ---
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# coco_cats = {}
# coco_cats_inv = {}

# def prep_coco_cats():
#     """ Prepare inverted table for category id lookup. """
#     if len(coco_cats) > 0: return
    
#     for coco_cat_id, transformed_cat_id_p1 in cfg.dataset.label_map.items():
#         transformed_cat_id = transformed_cat_id_p1 - 1
#         coco_cats[transformed_cat_id] = coco_cat_id
#         coco_cats_inv[coco_cat_id] = transformed_cat_id

# def get_coco_cat(transformed_cat_id):
#     return coco_cats[transformed_cat_id]


# class APDataObject:
#     """ Stores all the information necessary to calculate the AP for one IoU and one class. """
#     def __init__(self):
#         self.data_points = []
#         self.num_gt_positives = 0

#     def push(self, score:float, is_true:bool):
#         self.data_points.append((score, is_true))
    
#     def add_gt_positives(self, num_positives:int):
#         self.num_gt_positives += num_positives

#     def is_empty(self) -> bool:
#         return len(self.data_points) == 0 and self.num_gt_positives == 0

#     def get_ap(self) -> float:
#         """ Warning: result not cached. """
#         if self.num_gt_positives == 0:
#             return 0

#         self.data_points.sort(key=lambda x: -x[0])

#         precisions, recalls = [], []
#         num_true, num_false = 0, 0

#         for datum in self.data_points:
#             if datum[1]: num_true += 1
#             else: num_false += 1
            
#             precision = num_true / (num_true + num_false)
#             recall    = num_true / self.num_gt_positives

#             precisions.append(precision)
#             recalls.append(recall)

#         for i in range(len(precisions)-1, 0, -1):
#             if precisions[i] > precisions[i-1]:
#                 precisions[i-1] = precisions[i]

#         y_range = [0] * 101
#         x_range = np.array([x / 100 for x in range(101)])
#         recalls = np.array(recalls)

#         indices = np.searchsorted(recalls, x_range, side='left')
#         for bar_idx, precision_idx in enumerate(indices):
#             if precision_idx < len(precisions):
#                 y_range[bar_idx] = precisions[precision_idx]

#         return sum(y_range) / len(y_range)

# def calc_map(ap_data):
#     print('Calculating mAP...')
#     aps = [{'box': [], 'mask': []} for _ in iou_thresholds]

#     for _class in range(len(cfg.dataset.class_names)):
#         for iou_idx in range(len(iou_thresholds)):
#             for iou_type in ('box', 'mask'):
#                 ap_obj = ap_data[iou_type][iou_idx][_class]
#                 if not ap_obj.is_empty():
#                     aps[iou_idx][iou_type].append(ap_obj.get_ap())

#     all_maps = {'box': OrderedDict(), 'mask': OrderedDict()}

#     for iou_type in ('box', 'mask'):
#         all_maps[iou_type]['all'] = 0
#         for i, threshold in enumerate(iou_thresholds):
#             mAP = sum(aps[i][iou_type]) / len(aps[i][iou_type]) * 100 if len(aps[i][iou_type]) > 0 else 0
#             all_maps[iou_type][int(threshold*100)] = mAP
#         all_maps[iou_type]['all'] = (sum(all_maps[iou_type].values()) / (len(all_maps[iou_type].values())-1))
    
#     return all_maps

# def print_maps(all_maps):
#     make_row = lambda vals: (' %5s |' * len(vals)) % tuple(vals)
#     make_sep = lambda n:  ('-------+' * n)
#     print()
#     print(make_row([''] + [('.%d ' % x if isinstance(x, int) else x + ' ') for x in all_maps['box'].keys()]))
#     print(make_sep(len(all_maps['box']) + 1))
#     for iou_type in ('box', 'mask'):
#         print(make_row([iou_type] + ['%.2f' % x if x < 100 else '%.1f' % x for x in all_maps[iou_type].values()]))
#     print(make_sep(len(all_maps['box']) + 1))
#     print()


# iou_thresholds = [x / 100 for x in range(50, 100, 5)]

# print("Evaluation helper functions and classes defined.")
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [27]:
from layers.output_utils import postprocess
from eval import APDataObject, iou_thresholds, prep_coco_cats, evaluate, print_maps, calc_map
from data import COCODetection, get_label_map, MEANS, COLORS
from yolact import Yolact
from utils.augmentations import BaseTransform, FastBaseTransform, Resize
from utils.functions import MovingAverage, ProgressBar
from layers.box_utils import jaccard, center_size, mask_iou
from utils import timer
from utils.functions import SavePath
from layers.output_utils import postprocess, undo_image_transformation
import pycocotools

from data import cfg, set_cfg, set_dataset

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import argparse
import time
import random
import cProfile
import pickle
import json
import os
from collections import defaultdict
from pathlib import Path
from collections import OrderedDict
from PIL import Image

import matplotlib.pyplot as plt
import cv2

def evaluate(net:Yolact, dataset, device):
    net.eval()
    
    # Setup
    if cfg.dataset.label_map is None:
        cfg.dataset.label_map = {x+1: x+1 for x in range(len(cfg.dataset.class_names))}
    prep_coco_cats()
    
    ap_data = {
        'box' : [[APDataObject() for _ in cfg.dataset.class_names] for _ in iou_thresholds],
        'mask': [[APDataObject() for _ in cfg.dataset.class_names] for _ in iou_thresholds]
    }
    dataset_size = len(dataset)
    progress_bar = ProgressBar(30, dataset_size)
    
    print('\nEvaluating Performance...') # (I fixed a small typo here for you too)

    # --- THIS IS THE CRITICAL FIX ---
    # Wrap the entire evaluation loop in a no_grad context
    with torch.no_grad():
        try:
            for image_idx in range(len(dataset)):
                img, gt, gt_masks, h, w, num_crowd = dataset.pull_item(image_idx)
                
                batch = img.unsqueeze(0).to(device)
                preds = net(batch)

                # Perform postprocessing and metric preparation
                classes, scores, boxes, masks = postprocess(preds, w, h, score_threshold=cfg.nms_conf_thresh)
                
                if classes.size(0) != 0:
                    # Move everything to the CPU for metric calculation
                    classes, scores, boxes, masks = [x.cpu() for x in [classes, scores, boxes, masks]]

                    # Prepare ground truth
                    gt_boxes = torch.Tensor(gt[:, :4]); gt_boxes[:, [0, 2]] *= w; gt_boxes[:, [1, 3]] *= h
                    gt_classes = list(gt[:, 4].astype(int))
                    gt_masks = torch.Tensor(gt_masks).view(-1, h*w)

                    # The rest of this is the core logic from prep_metrics
                    num_pred, num_gt = len(classes), len(gt_classes)
                    mask_iou_cache = mask_iou(masks, gt_masks)
                    bbox_iou_cache = jaccard(boxes, gt_boxes)
                    
                    box_indices = sorted(range(num_pred), key=lambda i: -scores[i])
                    mask_indices = sorted(box_indices, key=lambda i: -scores[i])
                    
                    iou_types = [
                        ('box', lambda i,j: bbox_iou_cache[i, j].item(), scores, box_indices),
                        ('mask', lambda i,j: mask_iou_cache[i, j].item(), scores, mask_indices)
                    ]

                    for _class in set(gt_classes):
                        for iou_idx, iou_threshold in enumerate(iou_thresholds):
                            for iou_type, iou_func, score_func, indices in iou_types:
                                gt_used = [False] * num_gt
                                ap_obj = ap_data[iou_type][iou_idx][_class]
                                ap_obj.add_gt_positives(sum([1 for x in gt_classes if x == _class]))

                                for i in indices:
                                    if classes[i] != _class: continue
                                    max_iou_found, max_match_idx = iou_threshold, -1
                                    for j in range(num_gt):
                                        if gt_used[j] or gt_classes[j] != _class: continue
                                        iou = iou_func(i, j)
                                        if iou > max_iou_found: max_iou_found, max_match_idx = iou, j
                                    
                                    if max_match_idx >= 0:
                                        gt_used[max_match_idx] = True
                                        ap_obj.push(score_func[i], True)
                                    else:
                                        ap_obj.push(score_func[i], False)
                
                progress = (image_idx+1) / dataset_size * 100
                progress_bar.set_val(image_idx+1)
                print(f'\rProcessing Images  {repr(progress_bar)} {image_idx+1} / {dataset_size} ({progress:5.2f}%)', end='')

            # Note: This part should be outside the try...except but still inside the function
            # And also outside the with torch.no_grad() block
            
        except KeyboardInterrupt:
            print('Stopping...')
            return None

    # This calculation happens after the loop is done
    all_maps = calc_map(ap_data)
    return all_maps

In [28]:
# %%
# ========================================================================================
# MONKEY-PATCH CELL to Bypass Broken Cython NMS
# ========================================================================================
#
# This cell replaces the C-based NMS with a pure PyTorch version from torchvision.
# This avoids the Cython compilation error without needing to edit any original project files.

import torch
import torchvision
from layers.functions.detection import Detect

print("Applying in-notebook patch to bypass cython_nms.pyx compilation error...")

def patched_traditional_nms(self, boxes, masks, scores, iou_threshold, conf_thresh):
    """
    This function replaces the original traditional_nms function.
    It uses torchvision.ops.nms, which is a pure PyTorch implementation.
    """
    num_classes = scores.size(0)
    idx_lst = []
    cls_lst = []
    scr_lst = []

    # Do NMS for each class separately
    for _cls in range(num_classes):
        cls_scores = scores[_cls, :]
        conf_mask = cls_scores > conf_thresh
        
        if not conf_mask.any():
            continue

        # Filter by confidence threshold
        _scores = cls_scores[conf_mask]
        _boxes = boxes[conf_mask]
        
        # The core of the replacement: use torchvision's nms
        keep = torchvision.ops.nms(_boxes, _scores, iou_threshold)
        
        # Find original indices of kept boxes
        original_indices = torch.where(conf_mask)[0][keep]
        
        idx_lst.append(original_indices)
        cls_lst.extend([_cls] * len(keep))
        scr_lst.append(_scores[keep])

    if len(idx_lst) == 0:
        # Return empty tensors in the correct format if no detections are left
        return torch.empty(0, 4, device=boxes.device), \
               torch.empty(0, masks.shape[1], device=masks.device), \
               torch.empty(0, device=boxes.device, dtype=torch.long), \
               torch.empty(0, device=boxes.device)

    # Concatenate results from all classes
    idx = torch.cat(idx_lst, dim=0)
    classes = torch.as_tensor(cls_lst, device=boxes.device)
    scores = torch.cat(scr_lst, dim=0)
    
    # Sort by score across all classes
    scores, sort_idx = scores.sort(0, descending=True)
    idx = idx[sort_idx]
    classes = classes[sort_idx]

    # Enforce top_k limit
    if idx.size(0) > self.top_k:
        idx = idx[:self.top_k]
        classes = classes[:self.top_k]
        scores = scores[:self.top_k]

    return boxes[idx], masks[idx], classes, scores

# This is the actual monkey-patch: we replace the problematic method with our new one.
Detect.traditional_nms = patched_traditional_nms

print("Patch applied successfully. The model will now use the PyTorch NMS.")

Applying in-notebook patch to bypass cython_nms.pyx compilation error...
Patch applied successfully. The model will now use the PyTorch NMS.


In [29]:
# --- Load your trained model and run evaluation ---
from utils.augmentations import BaseTransform # FIX: Add this import statement

# Set the device based on the USE_CUDA flag from your config cell
device = torch.device('cuda' if USE_CUDA and torch.cuda.is_available() else 'cpu')
print(device)

# 1. Create the model and move it to the correct device
net = Yolact().to(device)

# 2. Load the weights you trained
# Make sure the path is correct (e.g., yolact_custom_final.pth)
trained_model_path = 'weights/yolact_custom_final.pth' 
net.load_weights(trained_model_path)
# net.load_weights(trained_model_path, device=device)
print(f"Loaded trained weights from: {trained_model_path}")

# 3. Create the validation dataset
val_dataset = COCODetection(image_path=cfg.dataset.valid_images,
                              info_file=cfg.dataset.valid_info,
                              transform=BaseTransform())

# 4. Prepare the COCO category mappings (THE FIX)
prep_coco_cats()

# 5. Run the evaluation
all_maps = evaluate(net, val_dataset, device)

# 6. Print the results
if all_maps:
    print_maps(all_maps)

cpu
Loaded trained weights from: weights/yolact_custom_final.pth
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating Performance...


[W907 07:21:10.420513681 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:21:10.421889710 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:21:10.422014693 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:21:10.422170834 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


Processing Images  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 1 / 170 ( 0.59%)

[W907 07:21:10.317626988 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:21:10.318479480 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:21:10.318636413 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:21:10.319252033 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


Processing Images  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 2 / 170 ( 1.18%)

...

Processing Images  █████████████████████████████░ 168 / 170 (98.82%)

[W907 07:23:23.544989177 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:23:23.545717326 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:23:23.545858290 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:23:23.545980398 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


Processing Images  █████████████████████████████░ 169 / 170 (99.41%)

[W907 07:23:23.018106250 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:23:23.018810034 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:23:23.018955045 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W907 07:23:23.019095829 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


Processing Images  ██████████████████████████████ 170 / 170 (100.00%)Calculating mAP...

       |  all  |  .50  |  .55  |  .60  |  .65  |  .70  |  .75  |  .80  |  .85  |  .90  |  .95  |
-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
   box |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |
  mask |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |
-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+


       |  all  |  .50  |  .55  |  .60  |  .65  |  .70  |  .75  |  .80  |  .85  |  .90  |  .95  |
-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
   box |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |
  mask |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |  0.00 |
-------+-------+-------+-------+----